# Imports

In [1]:
from ortools.sat.python import cp_model
from calendar import monthrange

from scheduling_funtions import *

# Creating Data

In [2]:
staff_list = [['Olivia', 0],
              ['Emma', 1], 
              ['Ava', 1],
              ['Charlotte', 1], 
              ['Sophia', 0],
              ['Amelia', 0],
              ['Isabella', 1],
              ['Mia', 1],
              ['Evelyn', 1],
              ['Harper', 0],
              ['Camila', 0],
              ['Gianna', 0],
              ['Abigail', 0], 
              ['Luna', 0],
              ['Ella', 0],
              ['Elizabeth', 0], 
              ['Sofia', 1],
              ['Emily', 1],
              ['Avery', 0],
              ['Mila', 1]]

staff = range(len(staff_list))

In [3]:
shift_list = ['0700 - 1500',
          '0730 - 1530 (FT)',
          '0930 - 1730',
          '1200 - 2000',
          '1400 - 2200',
          '1530 - 2330 (FT)',
          '1600 - 2400',
          '1800 - 0200',
          '2000 - 0400',
          '2200 - 0400',
          '2359 - 0700',
          'On call (22:00)']

shifts = range(len(shift_list))
midnight_shifts = shifts[6:]

In [4]:
planning_period = monthrange(2021, 5)
days = range(planning_period[1])

# Creating the model

In [5]:
model = cp_model.CpModel()

In [6]:
#decision variables
#staff 'm' works shift 's' on day 'd'
x = {(m, d, s) : \
    model.NewBoolVar('s%id%is%i' % (m, d, s)) \
    for m in staff \
    for d in days \
    for s in shifts}

In [7]:
#intermediate variables
#staff 'm' works on day 'd'
days_assigned = {(m, d) : \
    model.NewBoolVar('s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(days_assigned[(m,d)] == sum(x[(m,d,s)] for s in shifts))

# Constraints

In [9]:
# Maximum work 7 consecutive days
num_con_days = 7
for m in staff:
  for wind in window(days, num_con_days+1):
    constraint = [x[(m,d,s)] \
                  for d in wind \
                  for s in shifts]

    model.Add(sum(constraint) <= num_con_days)  

In [8]:
# All shifts should be taken by doctors
# No two doctors in the same shift on the same day
for d in days:
    for s in shifts:
      constraint = [x[(m,d,s)] \
                    for m in staff]
      model.Add(sum(constraint) == 1)

In [10]:
# 2 days off after 3 to 7 days of work in a row
# Defines an illegal pattern constraint
# 11101
for m in staff:
  for wind in window(days, 5):
    constraint = [days_assigned[(m,d)] \
                  for d in wind]

    model.AddForbiddenAssignments(constraint, [(1,1,1,0,1)])

In [11]:
# No two shifts same day
for m in staff:
  for d in days:
    constraint = [x[(m,d,s)] \
                  for s in shifts]
    model.Add(sum(constraint) <= 1)

In [12]:
# # 2 days off after last midnight (except on call shift). (Modified)
# for m in staff:
#   for wind in window(days, 3):
#     constraint = [x[(m,d,s)] \
#                 for d in wind \
#                 for s in midnight_shifts]

#     model.Add(sum(constraint) < )

In [13]:
# No midnights for staff in their first 6 months (need way to indicate when physician is in first 6 months of practice)

In [14]:
# # Maximum 2 midnights in a row (except for several physicians who only work midnights)
# for m in staff:
#   for wind in window(days, 3):
#     day1 = [x[(m,wind[0],s + midnight_offset)] \
#             for s in midnight_shifts]
#     day2 = [x[(m,wind[1],s + midnight_offset)] \
#             for s in midnight_shifts]
#     day3 = [x[(m,wind[2],s)].Not() \
#             for s in shifts]

#     model.Add(sum(day1 + day2 + day3) < 2 + len(day3))

In [15]:
# On call shift - day after rules

In [16]:
# # Certain physicians work only FT shift (0730,1530 shift)
# for m in staff:
#   constraint1 = [x[(m,d,shifts[0])] \
#                 for d in days]
#   constraint2 = [x[(m,d,s)] \
#                 for d in days \
#                 for s in shifts[2:4]]
#   constraint3 = [x[(m,d,s)] \
#                 for d in days \
#                 for s in shifts[6:]]

#   model.Add(sum(constraint1 + constraint2 + constraint3) * staff_list[m][1] < 1)

In [17]:
# No 2000, 2200, or midnight shift prior to day requested off

In [18]:
# Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.

# Solving the Model

In [19]:
# model.Maximize(0)

solver = cp_model.CpSolver()
solver.Solve(model)

for d in days:
    print('Day', d)
    for m in staff:
        for s in shifts:
            if solver.Value(x[(m, d, s)]) == 1:
              print(staff_list[m], 'works shift', shift_list[s])
    print()

Day 0
['Olivia', 0] works shift 0700 - 1500
['Emma', 1] works shift 1800 - 0200
['Charlotte', 1] works shift 0730 - 1530 (FT)
['Sophia', 0] works shift 1400 - 2200
['Amelia', 0] works shift 1530 - 2330 (FT)
['Mia', 1] works shift 1200 - 2000
['Evelyn', 1] works shift 2200 - 0400
['Gianna', 0] works shift On call (22:00)
['Luna', 0] works shift 1600 - 2400
['Sofia', 1] works shift 2359 - 0700
['Avery', 0] works shift 2000 - 0400
['Mila', 1] works shift 0930 - 1730

Day 1
['Olivia', 0] works shift 1800 - 0200
['Emma', 1] works shift 2000 - 0400
['Ava', 1] works shift 0930 - 1730
['Isabella', 1] works shift 0700 - 1500
['Mia', 1] works shift 1530 - 2330 (FT)
['Evelyn', 1] works shift 2200 - 0400
['Harper', 0] works shift On call (22:00)
['Gianna', 0] works shift 1600 - 2400
['Abigail', 0] works shift 0730 - 1530 (FT)
['Luna', 0] works shift 1400 - 2200
['Ella', 0] works shift 2359 - 0700
['Avery', 0] works shift 1200 - 2000

Day 2
['Olivia', 0] works shift On call (22:00)
['Ava', 1] works